In [60]:
import pandas as pd
import json
from itertools import chain

files = [
    # "js_2023_02_06/warm/query_responses.json",
    # "js_2023_02_06/cold/query_responses.json",
    # "merlin/cmi-cache-only/all-commands/main/full_responses.json",
    # # "merlin/cmi-cache-only/all-commands/merlin-domains/full_responses.json",
    # # "merlin/cmi-cache-only/complete-prefix/main/full_responses.json",
    # # "merlin/cmi-cache-only/complete-prefix/merlin-domains/full_responses.json",
    # "merlin/full-cache/all-commands/main/full_responses.json",
    # "merlin/full-cache/all-commands/merlin-domains/full_responses.json",
    # "irmin-15-10-2025/10/cmi-cache-only/all-commands/data-race-free-merlin-domains",
    # "irmin-15-10-2025/10/cmi-cache-only/all-commands/main",
    "ocaml-lsp-17-10-2025/src/10/cmi-cache-only/all-commands/data-race-free-merlin-domains/full_responses.json",
    "ocaml-lsp-17-10-2025/src/10/cmi-cache-only/all-commands/main/full_responses.json",
    "ocaml-lsp-17-10-2025/types/type-enclosing-only/cmi-cache-only/data-race-free-merlin-domains/full_responses.json",
    "ocaml-lsp-17-10-2025/types/type-enclosing-only/cmi-cache-only/main/full_responses.json",
    "ocaml-lsp-17-10-2025/types/type-enclosing-only/full-cache/data-race-free-merlin-domains/full_responses.json",
    "ocaml-lsp-17-10-2025/types/type-enclosing-only/full-cache/main/full_responses.json",
    "ocaml-lsp-17-10-2025/types/type-enclosing-only/no-cache/data-race-free-merlin-domains/full_responses.json",
    "ocaml-lsp-17-10-2025/types/type-enclosing-only/no-cache/main-merlin/full_responses.json",
]

files_perf_overview_cinaps = [
    ("ocaml-lsp-17-10-2025/src/10/cmi-cache-only/all-commands/main/full_responses.json", "All of OCaml-lsp. Mixed hot/cold cache."),
    ("ocaml-lsp-17-10-2025/types/cmi-cache-only/main/full_responses.json", "Huge file in OCaml-lsp. Mixed hot/cold cache"),
    # "ocaml-lsp-17-10-2025/types/no-cache/main/full_responses.json",
    # "ocaml-lsp-17-10-2025/types/type-enclosing-only/no-cache/main-merlin/full_responses.json",
    # "irmin-16-10-2025/cmi-cache-only/irmin/all-commands/main/full_responses.json",
]

files_perf_overview = [
    ("js_2023_02_06/cold/query_responses.json", "Big cold base. Cold cache"),
]


def frame_from_file(file):
    data = pd.read_json(file, lines = True)

    resp = list(chain.from_iterable(data['responses']))

    resp_with_id = []

    for _, row in data.iterrows():
        sample_id = row['sample_id']
        for resp in row['responses']:
            resp_copy = resp.copy()
            resp_copy['sample_id'] = sample_id
            resp_with_id.append(resp_copy)

    return pd.json_normalize(resp_with_id, sep='.')


In [61]:
def print_percentiles(frame, file, merge_ppx):
    percentiles = [0.25, 0.5, 0.75, 0.90, 0.95]


    desc_clock = frame['timing.clock'].describe(percentiles=percentiles)
    desc_reader = frame['timing.reader'].describe(percentiles=percentiles)
    desc_ppx = frame['timing.ppx'].describe(percentiles=percentiles)
    desc_typer = frame['timing.typer'].describe(percentiles=percentiles)
    desc_query = frame['timing.query'].describe(percentiles=percentiles)


    df_clock  = pd.DataFrame(desc_clock).T
    df_reader = pd.DataFrame(desc_reader).T
    df_ppx    = pd.DataFrame(desc_ppx).T
    df_typer  = pd.DataFrame(desc_typer).T
    df_query  = pd.DataFrame(desc_query).T


    df_clock.index  = ["total"]
    df_reader.index = ["parser"]
    df_ppx.index    = ["metaprogramming"]
    df_typer.index  = ["typer"]
    df_query.index  = ["query analysis"]

    # parts = file.split("/")
    # cache_part = parts[1]
    # cmd_part = parts[2]
    # branch_part = parts[3]
    # print(f"Branch: {branch_part}")
    # print(f"Cache: {cache_part}")
    # print(f"Commands: {cmd_part}")



    df = pd.concat([ df_clock, df_reader, df_ppx, df_typer, df_query])

    if merge_ppx:
            # Create the merged row
            merged_row = df.loc['parser'] + df.loc['metaprogramming']
            merged_row.name = 'parser + metaprogramming'

            # Drop the original rows if you want only the merged row
            df = df.drop(['parser', 'metaprogramming'])

            # Insert the merged row at position 1 (second row)
            df = pd.concat([df.iloc[:1], merged_row.to_frame().T, df.iloc[1:]])

    df = df.round(1)
    display(df[['mean', '95%']])
    print("\n\n")

for f, d in files_perf_overview:
    print(d)
    frame = frame_from_file(f)
    print_percentiles(frame, f, False)

for f, d in files_perf_overview_cinaps:
    print(d)
    frame = frame_from_file(f)
    print_percentiles(frame, f, True)

    





Big cold base. Cold cache


,mean,95%
total,2043.9,7934.0
parser,7.3,16.0
metaprogramming,48.9,66.0
typer,1562.5,4507.0
query analysis,412.7,3401.0





All of OCaml-lsp. Mixed hot/cold cache.


,mean,95%
total,79.9,95.5
parser + metaprogramming,17.1,18.0
typer,58.0,80.0
query analysis,4.1,13.0





Huge file in OCaml-lsp. Mixed hot/cold cache


,mean,95%
total,1527.4,1853.5
parser + metaprogramming,367.6,650.0
typer,1126.3,1424.5
query analysis,31.7,68.5
